In [3]:
%pip install vanna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\PedroGeraghty\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import vanna
from vanna.remote import VannaDefault
from dotenv import load_dotenv
import os

In [5]:
load_dotenv('.env', override=True)
VANNA_KEY = os.getenv('VANNA_KEY')
VANNA_MODEL_NAME = os.getenv('VANNA_MODEL_NAME')
DB_SERVER = os.getenv('DB_SERVER')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

In [6]:

vanna_model_name = VANNA_MODEL_NAME
vn = VannaDefault(model=vanna_model_name, api_key=VANNA_KEY)

# Training
You only need to train once. Do not train again unless you want to add more training data.

In [7]:
vn.connect_to_postgres(DB_SERVER,DB_NAME,DB_USER,DB_PASSWORD,DB_PORT)

# Remove training data

In [8]:
# id =  vn.add_ddl(f"""CREATE EXTENSION IF NOT EXISTS "uuid-ossp" WITH SCHEMA public;""")
# vn.remove_training_data(id)
# vn.remove_training_data("68738-sql")

# Provide DDL from dump sql file

In [31]:
import re

# Read the SQL file
with open('schemas_db.sql', 'r') as file:
    data = file.read()

# Regular expression to match PostgreSQL statements
pattern = r"(.*?;)"
matches = re.findall(pattern, data, re.DOTALL)

# Print all PostgreSQL statements
for match in matches:
    print(match)
    # vn.add_ddl(match)

--
-- PostgreSQL database dump
--

-- Dumped from database version 16.1 (Debian 16.1-1.pgdg120+1)
-- Dumped by pg_dump version 16.1 (Debian 16.1-1.pgdg120+1)

SET statement_timeout = 0;

SET lock_timeout = 0;

SET idle_in_transaction_session_timeout = 0;

SET client_encoding = 'UTF8';

SET standard_conforming_strings = on;

SELECT pg_catalog.set_config('search_path', '', false);

SET check_function_bodies = false;

SET xmloption = content;

SET client_min_messages = warning;

SET row_security = off;


--
-- Name: hr;
 Type: SCHEMA;
 Schema: -;
 Owner: postgres
--

CREATE SCHEMA hr;



ALTER SCHEMA hr OWNER TO postgres;


--
-- Name: humanresources;
 Type: SCHEMA;
 Schema: -;
 Owner: postgres
--

CREATE SCHEMA humanresources;



ALTER SCHEMA humanresources OWNER TO postgres;


--
-- Name: SCHEMA humanresources;
 Type: COMMENT;
 Schema: -;
 Owner: postgres
--

COMMENT ON SCHEMA humanresources IS 'Contains objects related to employees and departments.';



--
-- Name: pe;
 Type: SCHEMA;
 

# Check training data

In [9]:
vn.get_training_data()


,id,training_data_type,question,content
0,21244-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON COLUMN prod...
1,20609-ddl,ddl,NaN,Type: COMMENT;
2,21366-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON COLUMN prod...
3,21200-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON TABLE produ...
4,20731-ddl,ddl,NaN,Schema: person;
...,...,...,...,...
3804,21201-ddl,ddl,NaN,\n\n\n--\n-- Name: COLUMN product.productid;
3805,20476-ddl,ddl,NaN,Type: VIEW;
3806,20349-ddl,ddl,NaN,\n\n\n--\n-- Name: COLUMN employee.birthdate;
3807,20914-ddl,ddl,NaN,Type: COMMENT;


# Provide documentation on information schema

In [11]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [12]:
vn.get_training_data()

,id,training_data_type,question,content
0,21244-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON COLUMN prod...
1,20609-ddl,ddl,NaN,Type: COMMENT;
2,21366-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON COLUMN prod...
3,21200-ddl,ddl,NaN,Owner: postgres\n--\n\nCOMMENT ON TABLE produ...
4,20731-ddl,ddl,NaN,Schema: person;
...,...,...,...,...
4168,21201-ddl,ddl,NaN,\n\n\n--\n-- Name: COLUMN product.productid;
4169,20476-ddl,ddl,NaN,Type: VIEW;
4170,20349-ddl,ddl,NaN,\n\n\n--\n-- Name: COLUMN employee.birthdate;
4171,20914-ddl,ddl,NaN,Type: COMMENT;
